## 1. Ответьте на вопросы:
про линейную регрессию на мой взгляд неплохо описанно [тут](https://habr.com/ru/articles/804135/)

In [138]:
from google.colab import drive
drive.mount('/content/drive')

!pip uninstall scikit-learn -y

!pip install -U scikit-learn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found existing installation: scikit-learn 1.5.0
Uninstalling scikit-learn-1.5.0:
  Successfully uninstalled scikit-learn-1.5.0
  Using cached scikit_learn-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)


### 1.1. Получить аналитическое решение задачи регресии используя векторную форму уравнения
Для решения задачи регресии могут быть исолпьзованны несколько способов(аналитическое решение, градиентный спуск и его разновидности).

Аналитическое решение:
На мой взгляд аналитическое решение неплохо описанно [тут](https://habr.com/ru/articles/474602/). Чтобы получить аналитическое решение задачи регрессии с использованием векторной формы уравнения, мы следуем шагам, изложенным в предоставленных источниках. Цель состоит в том, чтобы найти оптимальные веса (`w`) и смещение (`b`), которые минимизируют функцию стоимости, которая представляет собой сумму квадратов разностей между прогнозируемыми и фактическими значениями.
В отличие от большинства моделей, регрессия представляет собой удивительно простую задачу оптимизации. В частности, мы можем найти оптимальные параметры (оцененные на основе обучающих данных) аналитически, применив следующую простую формулу.
Во-первых мы можем отнести смещение(bias) `b` в параметр веса `w` путем добовления столбца к матрице состоящего из единиц.
ТОгда задача прогнозирования состоит в том, чтобы минимизировать ошибку среднеквадратичную ||y - Xw||^2. Затем нам нужно будет найти производную от потери по `w` и установить его равным нулю:

dw||y - Xw||^2 = 2XT(Xw-y) = 0 и XTy = XTXw
w = (XTX)^-1XTy

### 1.2. Что изменится в решении при добавлении к функции потерь регуляризации L1 и L2.
Добавление регуляризации делает модель менее переобученой - более простой. Добалвение регуляризации добавлет штрафы при расчете весов (те есть регуляризации ограничивает слишком большие параметры)

### 1.3. Объясните, почему регуляризация L1 обычно используется для выбора функций. Почему после установки модели много весов, равных 0?
Регуляризация L1, также известная как «Лассо» (оператор наименьшего абсолютного сжатия и выбора), обычно используется для выбора функций в моделях машинного обучения из-за ее способности автоматически выполнять критерии отбора. Это за счет добавления штрафного термина к функциям потерь, которые пропорциональны абсолютным значениям коэффициентов модели. В результате некоторые коэффициенты могут стать равными внешними факторами, что означает, что модель будет внешним признаком. Этот процесс помогает сосредоточиться на наиболее важных признаках, улучшить интерпретируемость модели и снизить риск переобучения

### 1.4. Объясните, как можно использовать одни и те же модели (линейная регрессия, ридж и т. д.), но при этом обеспечить возможность соответствия нелинейным зависимостям?
Для обеспечения возможностей соответствия нелинейным зависимостям в моделях, таких как линейная регрессия, ридж-регрессия и т.д., можно использовать несколько подходов:
-
Показатели преобразования : Показатели преобразования могут включать в себя логарифмирование, квадратирование, кубирование и другие функции, которые могут помочь в выявлении нелинейных зависимостей. Это позволяет моделям лучше адаптироваться к нелинейным закономерностям в данных- .
Полиномиальные признаки : Создание полиномиальных признаков на основе исходных данных может помочь моделировать нелинейные зависимости. Например, если исходный признак (x) имеет полиномиальную зависимость от переменной переменной, то можно создать новые признаки (x^2), (x^3) и т.д., чтобы можно было смоделировать эту нелинейность.

## 2. Introduction - make all preprocess staff from the previous lesson
- Import libraries.
- Read train and test parts.
- Preprocess “interest level” feature.

In [186]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import scipy
import statsmodels
import seaborn as sns
import zipfile
import json
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
# from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [140]:
d = None
data = None
# with zipfile.ZipFile("data/train.json.zip", "r") as z:
with zipfile.ZipFile("/content/drive/MyDrive/ML/ML.Project_2-1/src/data/train.json.zip", "r") as z:
    for filename in z.namelist():
        with z.open(filename) as f:
            data = f.read()
            d = json.loads(data.decode("utf-8"))

df = pd.DataFrame.from_dict(d)
df.head(3)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium


In [141]:
# Оставляем пока только нужные столбцы
df = df[['bathrooms',
         'features',
         'bedrooms',
         'price',
         'interest_level']].reset_index()

# interest _level переводим в категориальные данные
categorical_data = {'low': 0, 'medium':1, 'high':2}
df['interest_level'] = df['interest_level'].map(categorical_data)
df.head()

,index,bathrooms,features,bedrooms,price,interest_level
0,4,1.0,"[Dining Room, Pre-War, Laundry in Building, Di...",1,2400,1
1,6,1.0,"[Doorman, Elevator, Laundry in Building, Dishw...",2,3800,0
2,9,1.0,"[Doorman, Elevator, Laundry in Building, Laund...",2,3495,1
3,10,1.5,[],3,3000,1
4,15,1.0,"[Doorman, Elevator, Fitness Center, Laundry in...",0,2795,0


## 3. Intro data analysis part 2

- Lets generate additional features for better model quality. Consider a column named “features”. It consists of a list of highlights of the current flat.
- Remove unused symbols ([,],’,” and space) from the column.

In [142]:
def removing(string):
    temp = ','.join(string)
    temp = temp.replace(' ', '')
    return temp

df['features'] = df['features'].apply(lambda x: removing(x))
df.head()

,index,bathrooms,features,bedrooms,price,interest_level
0,4,1.0,"DiningRoom,Pre-War,LaundryinBuilding,Dishwashe...",1,2400,1
1,6,1.0,"Doorman,Elevator,LaundryinBuilding,Dishwasher,...",2,3800,0
2,9,1.0,"Doorman,Elevator,LaundryinBuilding,LaundryinUn...",2,3495,1
3,10,1.5,,3,3000,1
4,15,1.0,"Doorman,Elevator,FitnessCenter,LaundryinBuilding",0,2795,0


- Split values in every row with separator “,” and collect the result in one huge list for the whole dataset. You may use DataFrame.iterrows().

In [143]:
huge_list = []
for _, row in df.iterrows():
    temp = list(row['features'].split(','))
    if len(temp) < 2:
        continue
    else:
        huge_list.extend(temp)
len(huge_list)

263566

- How many unique values consist of a result list?

In [144]:
print('Количество уникальных элементов в списке: ', len(list(set(huge_list))))

Количество уникальных элементов в списке:  1482


- Let's get to know the new library - Collections. With this package you could effectively get quantity statistics about your data.
- Count the most popular features of our huge list and take the top 20 for this moment.
- If everything correct you should get next values: 'Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed', 'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding', 'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck', 'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony', 'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace'.

In [145]:
uniq = pd.DataFrame(huge_list, columns=['unique'])

In [146]:
uniq = uniq['unique'].value_counts()

In [147]:
uniq.head(20)

unique
Elevator             25408
CatsAllowed          23261
HardwoodFloors       22477
DogsAllowed          22029
Doorman              20611
Dishwasher           20364
NoFee                17766
LaundryinBuilding    16263
FitnessCenter        13214
LaundryinUnit         8728
Pre-War               8211
RoofDeck              6525
OutdoorSpace          5266
DiningRoom            5135
HighSpeedInternet     4298
Balcony               2991
SwimmingPool          2730
LaundryInBuilding     2551
NewConstruction       2549
Terrace               2282
Name: count, dtype: int64

- Now generate 20 new features based on top 20 values: 1 if value in “feature” column else 0.
- Extend our features set with 'bathrooms', 'bedrooms', 'interest_level' and lets create a special variable feature_list with all feature names. Now there are 23 values. All models should be trained on these 23 features.

In [148]:
# Получаем название фичей в виде списка
lst = uniq.head(20)
lst = lst.index.to_list()
lst

['Elevator',
 'CatsAllowed',
 'HardwoodFloors',
 'DogsAllowed',
 'Doorman',
 'Dishwasher',
 'NoFee',
 'LaundryinBuilding',
 'FitnessCenter',
 'LaundryinUnit',
 'Pre-War',
 'RoofDeck',
 'OutdoorSpace',
 'DiningRoom',
 'HighSpeedInternet',
 'Balcony',
 'SwimmingPool',
 'LaundryInBuilding',
 'NewConstruction',
 'Terrace']

In [149]:
# Добавляем новые признаки и пока заполняем нулями
for feature in lst:
    df[feature] = 0
df.head()

,index,bathrooms,features,bedrooms,price,interest_level,Elevator,CatsAllowed,HardwoodFloors,DogsAllowed,...,Pre-War,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
0,4,1.0,"DiningRoom,Pre-War,LaundryinBuilding,Dishwashe...",1,2400,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,1.0,"Doorman,Elevator,LaundryinBuilding,Dishwasher,...",2,3800,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,1.0,"Doorman,Elevator,LaundryinBuilding,LaundryinUn...",2,3495,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,1.5,,3,3000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,1.0,"Doorman,Elevator,FitnessCenter,LaundryinBuilding",0,2795,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
# Выставялем значение 1 если данный признак есть в столбце features
for index, row in df.iterrows():
    for i in lst:
        if i in row['features']:
            df.at[index, i] = 1

del df['features']
df.head()

,index,bathrooms,bedrooms,price,interest_level,Elevator,CatsAllowed,HardwoodFloors,DogsAllowed,Doorman,...,Pre-War,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
0,4,1.0,1,2400,1,0,1,1,1,0,...,1,0,0,1,0,0,0,0,0,0
1,6,1.0,2,3800,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,9,1.0,2,3495,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,10,1.5,3,3000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,1.0,0,2795,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49352 entries, 0 to 49351
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              49352 non-null  object 
 1   bathrooms          49352 non-null  float64
 2   bedrooms           49352 non-null  int64  
 3   price              49352 non-null  int64  
 4   interest_level     49352 non-null  int64  
 5   Elevator           49352 non-null  int64  
 6   CatsAllowed        49352 non-null  int64  
 7   HardwoodFloors     49352 non-null  int64  
 8   DogsAllowed        49352 non-null  int64  
 9   Doorman            49352 non-null  int64  
 10  Dishwasher         49352 non-null  int64  
 11  NoFee              49352 non-null  int64  
 12  LaundryinBuilding  49352 non-null  int64  
 13  FitnessCenter      49352 non-null  int64  
 14  LaundryinUnit      49352 non-null  int64  
 15  Pre-War            49352 non-null  int64  
 16  RoofDeck           493

## 4. Models implementation - linear regression

подготовка данных: делим на тренировочный и тестовый датасеты

In [152]:
X = df.drop(labels=['price', 'index'], axis=1)
y = df[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

- Implement python class for a linear regression algorithm with two basic methods - fit and predict. Use stochastic gradient descent to find optimal weights of the model. For better understanding we recommend additionally implementing separate versions of the algorithm with the analytical solution and non-stochastic gradient descent under the hood.

In [153]:
##
class myAnaliticLinearRegression:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

    # Вычисление для fit
    # Theta = (x^T * x)^-1 * x^T * y
    def _compute_normal_equation(self,X,y):
        X_temp = X.copy()
        # Изначально X - 39481*23
        # Сначала добавляем один столбец c единицами в X (смещение) получаем 39481*24:
        X_temp.insert(0, 'ones', np.ones((X_temp.shape[0], 1)))
        # 1. Умножаем X на транспонированный X получим матрицу 23*23:
        temp = X_temp.T.dot(X_temp)
        # 2. находим обратную матрицу получим матрицу 23*23:
        temp = np.linalg.inv(temp)
        # 3. умножим на транспонированную X получим 23*39481:
        temp = temp.dot(X_temp.T)
        # 4. умножим на целевые значения y(39481*1) получим ТЕТА 23*1:
        theta = temp.dot(y)
        # intercept_ - 0 значение веткора theta
        self.intercept_ = theta[0]
        # coef_ - остальные значения веткора theta
        self.coef_ = theta[1:].T

    def predict(self,X):
        return pd.DataFrame(np.dot(X,self.coef_[0]) + self.intercept_).to_numpy()

    def fit(self,X,y):
        self._compute_normal_equation(X, y)


In [154]:
class myGDLinearRegression():
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

        # функция градиента
    def gr_mserror_mat(self, X, w, y):
        y_pred = X @ w
        return 2/len(X)*(y - y_pred) @ (-X)

    def fit(self, X, y, learning_rate=0.1, num_iterations=1000, penalty=None, eps = 0.0001):
        X_temp = X.copy()
        y_temp = y['price']
        # Сначала добавляем один столбец c единицами в X (смещение) получаем 39481*24:
        X_temp.insert(0, 'ones', np.ones((X_temp.shape[0], 1)))

        weights = np.zeros(X_temp.shape[1])
        next_weights = weights
        for i in range(num_iterations):
            cur_weights = next_weights
            next_weights = cur_weights - learning_rate * self.gr_mserror_mat(X_temp, cur_weights, y_temp)

            if np.linalg.norm(cur_weights - next_weights, ord=2) <= eps:
                print(i)
                break
        self.coef_ = pd.DataFrame(next_weights[1:]).to_numpy().T
        self.intercept_ = pd.DataFrame(next_weights).to_numpy()[0]

    def predict(self,X):
        return pd.DataFrame(np.dot(X,self.coef_[0]) + self.intercept_).to_numpy()

In [155]:
class mySGDLinearRegression():
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

        # функция градиента
    def gr_mserror_mat(self, X, w, y):
        y_pred = X @ w
        return 2/len(X)*(y - y_pred) @ (-X)

    def fit(self, X, y, learning_rate=0.001, num_iterations=5000, penalty=None, eps = 0.001):
        X_temp = X.copy()
        y_temp = y['price']
        # Сначала добавляем один столбец c единицами в X (смещение) получаем 39481*24:
        X_temp.insert(0, 'ones', np.ones((X_temp.shape[0], 1)))

        weights = np.zeros(X_temp.shape[1])
        next_weights = weights
        for i in range(num_iterations):
            idx = np.random.choice(len(X_temp))
            cur_weights = next_weights
            next_weights = cur_weights - learning_rate * self.gr_mserror_mat(X_temp.iloc[[idx]], cur_weights, y_temp.iloc[[idx]])

            if np.linalg.norm(cur_weights - next_weights, ord=2) <= eps:
                break
        self.coef_ = pd.DataFrame(next_weights[1:]).to_numpy().T
        self.intercept_ = pd.DataFrame(next_weights).to_numpy()[0]

    def predict(self,X):
        return pd.DataFrame(np.dot(X,self.coef_[0]) + self.intercept_).to_numpy()

- Give definition for R squared (R2) coefficient and implement function for calculation.

коэффицент корреляции может принимать значение от -1 до 1, если ноль, то корреляция отсутствует. на наших данных (предсказанных и реальных) данный параметр говорит насколько хорошо мы предсказали данные

In [156]:
def myMSE(y_pred, y_test):
    temp = pd.DataFrame(y_test)
    temp['pred'] = y_pred
    temp['result'] = (temp['price'] - temp['pred']) ** 2
    result = temp['result'].sum() / len(y_pred)
    return result ** 0.5

def myr2(y_pred, y_test):
    temp = pd.DataFrame(y_test)
    temp['pred'] = y_pred
    y_mean = temp['price'].mean()
    temp['mid'] = (temp['price'] - y_mean) ** 2
    temp['mse'] = pow((temp['price'] - temp['pred']),2)
    ssres = temp['mse'].sum()
    sstotal = temp['mid'].sum()
    result = 1-(ssres/sstotal)
    return result

def myMAE(y_pred, y_test):
    temp = pd.DataFrame(y_test)
    temp['pred'] = y_pred
    temp['abs_residue'] = abs(temp['price'] - temp['pred'])
    result = temp['abs_residue'].sum()/len(y_pred)
    return result

- Make predictions with your algorithm and estimate model with MAE, RMSE and R2 metrics.
- Initialize LinearRegression() from sklearn.linear_model, fit model and predict train and test parts, same as previous lesson.
- Compare quality metrics and make sure that difference is small. (Between your implementations and sklearn).
- Save metrics as in the previous lesson in a table with columns model, train, test for MAE table, RMSE table and R2 coefficient.


In [157]:
result_MAE = pd.DataFrame(columns=['model_MAE', 'train', 'test']) # Средняя абсолютная ошибка
result_RMSE = pd.DataFrame(columns=['model_RMSE', 'train', 'test']) # Корень среднеквадратичной ошибки
result_R2 = pd.DataFrame(columns=['model_R2', 'train', 'test']) # Корень среднеквадратичной ошибки

In [158]:
linreg = myAnaliticLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myMAE(y_train_predict, y_train)
mae_test = myMAE(y_test_predict, y_test)
result_MAE.loc[len(result_MAE.index)] = ['myAnaliticLinearRegression', mae_train, mae_test]

linreg = myGDLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myMAE(y_train_predict, y_train)
mae_test = myMAE(y_test_predict, y_test)
result_MAE.loc[len(result_MAE.index)] = ['myGDLinearRegression', mae_train, mae_test]

linreg = mySGDLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myMAE(y_train_predict, y_train)
mae_test = myMAE(y_test_predict, y_test)
result_MAE.loc[len(result_MAE.index)] = ['mySGDLinearRegression', mae_train, mae_test]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = metrics.mean_absolute_error(y_train, y_train_predict)
mae_test = metrics.mean_absolute_error(y_test, y_test_predict)
result_MAE.loc[len(result_MAE.index)] = ['sklearnLinearRegression', mae_train, mae_test]

result_MAE

,model_MAE,train,test
0,myAnaliticLinearRegression,1229.309643,1028.484374
1,myGDLinearRegression,1229.200764,1028.315910
2,mySGDLinearRegression,1123.739557,917.855819
3,sklearnLinearRegression,1229.309643,1028.484374


In [159]:
linreg = myAnaliticLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myMSE(y_train_predict, y_train)
mae_test = myMSE(y_test_predict, y_test)
result_RMSE.loc[len(result_RMSE.index)] = ['myAnaliticLinearRegression', mae_train, mae_test]

linreg = myGDLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myMSE(y_train_predict, y_train)
mae_test = myMSE(y_test_predict, y_test)
result_RMSE.loc[len(result_RMSE.index)] = ['myGDLinearRegression', mae_train, mae_test]

linreg = mySGDLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myMSE(y_train_predict, y_train)
mae_test = myMSE(y_test_predict, y_test)
result_RMSE.loc[len(result_RMSE.index)] = ['mySGDLinearRegression', mae_train, mae_test]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = metrics.root_mean_squared_error(y_train, y_train_predict)
mae_test = metrics.root_mean_squared_error(y_test, y_test_predict)
result_RMSE.loc[len(result_RMSE.index)] = ['sklearnLinearRegression', mae_train, mae_test]

result_RMSE

,model_RMSE,train,test
0,myAnaliticLinearRegression,24565.181235,2175.129388
1,myGDLinearRegression,24565.181401,2175.071869
2,mySGDLinearRegression,24576.992389,2155.258782
3,sklearnLinearRegression,24565.181235,2175.129388


In [160]:
linreg = myAnaliticLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myr2(y_train_predict, y_train)
mae_test = myr2(y_test_predict, y_test)
result_R2.loc[len(result_R2.index)] = ['myAnaliticLinearRegression', mae_train, mae_test]

linreg = myGDLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myr2(y_train_predict, y_train)
mae_test = myr2(y_test_predict, y_test)
result_R2.loc[len(result_R2.index)] = ['myGDLinearRegression', mae_train, mae_test]

linreg = mySGDLinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = myr2(y_train_predict, y_train)
mae_test = myr2(y_test_predict, y_test)
result_R2.loc[len(result_R2.index)] = ['mySGDLinearRegression', mae_train, mae_test]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_train_predict = linreg.predict(X_train)
y_test_predict = linreg.predict(X_test)
mae_train = metrics.r2_score(y_train, y_train_predict)
mae_test = metrics.r2_score(y_test, y_test_predict)
result_R2.loc[len(result_R2.index)] = ['sklearnLinearRegression', mae_train, mae_test]

result_R2

,model_R2,train,test
0,myAnaliticLinearRegression,0.005668,0.337366
1,myGDLinearRegression,0.005668,0.337401
2,mySGDLinearRegression,0.004991,0.382029
3,sklearnLinearRegression,0.005668,0.337366


## 5. Regularized models implementation - Ridge, Lasso, ElasticNet

- Implement Ridge, Lasso, ElasticNet algorithms: extend loss function with L2, L1 and both regularization correspondingly

In [161]:
class MyRidge() :
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        # переводим DataFrame в массивы numpy
        X = X.to_numpy()
        y = y.to_numpy()
        # Добавляем дополнительныю коллонку единиц
        X = np.hstack((np.ones((X.shape[0], 1)), X))

        # Общее уравнение имеет вид:
        # (X^T * X + alpha * I)^-1 * X^T * y
        I = np.eye(X.shape[1])
        # print(I[0,0])
        I[0, 0] = 0 # Exclude the intercept term from regularization
        self.coef_ = np.linalg.inv(X.T @ X + self.alpha * I) @ X.T @ y

        # Store the intercept term separately
        # print(self.coef_[0])
        self.intercept_ = self.coef_[0]
        self.coef_ = self.coef_[1:]

    def predict(self, X):
        X = X.to_numpy()
        # Apply the coefficients to predict the target values
        return X @ self.coef_ + self.intercept_

In [162]:
class MyLasso:
  def __init__(self, n_cycles=1000, eps = 0.0001, alpha = 0.03 ) -> None:
    self.w = 0
    self.n_cycles = n_cycles
    self.eps = eps
    self.alpha= alpha

  def fit(self, X, Y) -> None:
    alpha = self.alpha # шаг обучения
    lm = 0.3
    amount_samples, amount_features = X.shape
    w = np.random.normal(size=amount_features)
    np_soft_sign = np.vectorize(lambda x, eps=1e-7: np.sign(x) if abs(x) > eps else x / eps)
    for _ in range(self.n_cycles):
      signw = np_soft_sign(self.w)
      f = X.dot(w)
      err = f - np.array(Y.iloc[:,0])
      grad = 2 * X.T.dot(err)/amount_samples + lm*signw
      if (w.max()>1e25) or (np.absolute(alpha * grad).sum() < self.eps):
        break
      w -= alpha * grad
    self.w = w

  def predict(self, X) -> pd.DataFrame:
    return pd.DataFrame(X.dot(self.w)).to_numpy().reshape(-1,1)

In [163]:
class MyElasticNet:
  def __init__(self, l1_ratio = 0.5, n_cycles=500, print_eps=False, eps=0.1, alpha = 0.03) -> None:
    self.w = 0
    self.l1_ratio = l1_ratio
    self.n_cycles = n_cycles
    self.print_eps = print_eps
    self.eps = eps
    self.alpha= alpha

  def fit(self, X, Y) -> None:
    alpha = self.alpha # шаг обучения
    lm = 0.3 # лямбда, коэф забывания, для л1
    amount_samples, amount_features = X.shape
    w = np.random.normal(size=amount_features)
    np_soft_sign = np.vectorize(lambda x, eps=1e-7: np.sign(x) if abs(x) > eps else x / eps)
    for _ in range(self.n_cycles):
      signw = np_soft_sign(self.w)
      f = X.dot(w)
      err = f - np.array(Y.iloc[:,0])
      grad = 2 * X.T.dot(err)/amount_samples + (1-self.l1_ratio)*w + (self.l1_ratio)*lm*signw
      if (w.max()>1e25) or (np.absolute(alpha * grad).sum() < self.eps):
        break
      elif self.print_eps:
        print(np.absolute(alpha * grad).sum())
      w -= alpha * grad
    self.w = w
  def predict(self, X) -> pd.DataFrame:
    return pd.DataFrame(X.dot(self.w)).to_numpy().reshape(-1,1)

- Make predictions with your algorithm and estimate the model with MAE, RMSE and R2 metrics.
- Initialize Ridge(), Lasso() and ElasticNet() from sklearn.linear_model, fit model and make prediction for train and test samples, same as previous lesson.
- Compare quality metrics and make sure that difference is small. (Between your implementations and sklearn).
- Save metrics as in the previous lesson in a table with columns model, train, test for MAE table, RMSE table and R2 coefficient.

In [164]:
l2 = MyRidge()
l2.fit(X_train, y_train)
y_train_predict = l2.predict(X_train)
y_test_predict = l2.predict(X_test)
mae_train = myMAE(y_train_predict, y_train)
mae_test = myMAE(y_test_predict, y_test)
result_MAE.loc[len(result_MAE.index)] = ['myRidge', mae_train, mae_test]

l2 = Ridge()
l2.fit(X_train, y_train)
y_train_predict = l2.predict(X_train)
y_test_predict = l2.predict(X_test)
mae_train = metrics.mean_absolute_error(y_train, y_train_predict)
mae_test = metrics.mean_absolute_error(y_test, y_test_predict)
result_MAE.loc[len(result_MAE.index)] = ['sklearnRidge', mae_train, mae_test]


l1 = MyLasso()
l1.fit(X_train, y_train)
y_train_predict = l1.predict(X_train)
y_test_predict = l1.predict(X_test)
mae_train = myMAE(y_train_predict, y_train)
mae_test = myMAE(y_test_predict, y_test)
result_MAE.loc[len(result_MAE.index)] = ['myLasso', mae_train, mae_test]

l1 = Lasso()
l1.fit(X_train, y_train)
y_train_predict = l1.predict(X_train)
y_test_predict = l1.predict(X_test)
mae_train = metrics.mean_absolute_error(y_train, y_train_predict)
mae_test = metrics.mean_absolute_error(y_test, y_test_predict)
result_MAE.loc[len(result_MAE.index)] = ['sklearnLasso', mae_train, mae_test]


elastic = MyElasticNet()
elastic.fit(X_train, y_train)
y_train_predict = elastic.predict(X_train)
y_test_predict = elastic.predict(X_test)
mae_train = myMAE(y_train_predict, y_train)
mae_test = myMAE(y_test_predict, y_test)
result_MAE.loc[len(result_MAE.index)] = ['myElastic', mae_train, mae_test]

elastic = ElasticNet()
elastic.fit(X_train, y_train)
y_train_predict = elastic.predict(X_train)
y_test_predict = elastic.predict(X_test)
mae_train = metrics.mean_absolute_error(y_train, y_train_predict)
mae_test = metrics.mean_absolute_error(y_test, y_test_predict)
result_MAE.loc[len(result_MAE.index)] = ['sklearnElastic', mae_train, mae_test]

result_MAE

,model_MAE,train,test
0,myAnaliticLinearRegression,1229.309643,1028.484374
1,myGDLinearRegression,1229.200764,1028.315910
2,mySGDLinearRegression,1123.739557,917.855819
3,sklearnLinearRegression,1229.309643,1028.484374
4,myRidge,1229.240889,1028.408325
5,sklearnRidge,1229.240889,1028.408325
6,myLasso,1226.315987,1025.342315
7,sklearnLasso,1225.253433,1023.900827
8,myElastic,1108.552334,900.520436
9,sklearnElastic,1150.604032,934.382583


In [165]:
l2 = MyRidge()
l2.fit(X_train, y_train)
y_train_predict = l2.predict(X_train)
y_test_predict = l2.predict(X_test)
mae_train = myMSE(y_train_predict, y_train)
mae_test = myMSE(y_test_predict, y_test)
result_RMSE.loc[len(result_RMSE.index)] = ['myRidge', mae_train, mae_test]

l2 = Ridge()
l2.fit(X_train, y_train)
y_train_predict = l2.predict(X_train)
y_test_predict = l2.predict(X_test)
mae_train = metrics.root_mean_squared_error(y_train, y_train_predict)
mae_test = metrics.root_mean_squared_error(y_test, y_test_predict)
result_RMSE.loc[len(result_RMSE.index)] = ['sklearnRidge', mae_train, mae_test]


l1 = MyLasso()
l1.fit(X_train, y_train)
y_train_predict = l1.predict(X_train)
y_test_predict = l1.predict(X_test)
mae_train = myMSE(y_train_predict, y_train)
mae_test = myMSE(y_test_predict, y_test)
result_RMSE.loc[len(result_RMSE.index)] = ['myLasso', mae_train, mae_test]

l1 = Lasso()
l1.fit(X_train, y_train)
y_train_predict = l1.predict(X_train)
y_test_predict = l1.predict(X_test)
mae_train = metrics.root_mean_squared_error(y_train, y_train_predict)
mae_test = metrics.root_mean_squared_error(y_test, y_test_predict)
result_RMSE.loc[len(result_RMSE.index)] = ['sklearnLasso', mae_train, mae_test]


elastic = MyElasticNet()
elastic.fit(X_train, y_train)
y_train_predict = elastic.predict(X_train)
y_test_predict = elastic.predict(X_test)
mae_train = myMSE(y_train_predict, y_train)
mae_test = myMSE(y_test_predict, y_test)
result_RMSE.loc[len(result_RMSE.index)] = ['myElastic', mae_train, mae_test]

elastic = ElasticNet()
elastic.fit(X_train, y_train)
y_train_predict = elastic.predict(X_train)
y_test_predict = elastic.predict(X_test)
mae_train = metrics.root_mean_squared_error(y_train, y_train_predict)
mae_test = metrics.root_mean_squared_error(y_test, y_test_predict)
result_RMSE.loc[len(result_RMSE.index)] = ['sklearnElastic', mae_train, mae_test]

result_RMSE

,model_RMSE,train,test
0,myAnaliticLinearRegression,24565.181235,2175.129388
1,myGDLinearRegression,24565.181401,2175.071869
2,mySGDLinearRegression,24576.992389,2155.258782
3,sklearnLinearRegression,24565.181235,2175.129388
4,myRidge,24565.181236,2175.091064
5,sklearnRidge,24565.181236,2175.091064
6,myLasso,24565.228265,2173.152569
7,sklearnLasso,24565.189431,2172.939434
8,myElastic,24574.694627,2160.532264
9,sklearnElastic,24584.196502,2243.735846


In [166]:
l2 = MyRidge()
l2.fit(X_train, y_train)
y_train_predict = l2.predict(X_train)
y_test_predict = l2.predict(X_test)
rmse_train = myr2(y_train_predict, y_train)
rmse_test = myr2(y_test_predict, y_test)
result_R2.loc[len(result_R2.index)] = ['myRidge', rmse_train, rmse_test]

l2 = Ridge()
l2.fit(X_train, y_train)
y_train_predict = l2.predict(X_train)
y_test_predict = l2.predict(X_test)
rmse_train = metrics.r2_score(y_train, y_train_predict)
rmse_test = metrics.r2_score(y_test, y_test_predict)
result_R2.loc[len(result_R2.index)] = ['sklearnRidge', rmse_train, rmse_test]


l1 = MyLasso()
l1.fit(X_train, y_train)
y_train_predict = l1.predict(X_train)
y_test_predict = l1.predict(X_test)
rmse_train = myr2(y_train_predict, y_train)
rmse_test = myr2(y_test_predict, y_test)
result_R2.loc[len(result_R2.index)] = ['myLasso', rmse_train, rmse_test]

l1 = Lasso()
l1.fit(X_train, y_train)
y_train_predict = l1.predict(X_train)
y_test_predict = l1.predict(X_test)
rmse_train = metrics.r2_score(y_train, y_train_predict)
rmse_test = metrics.r2_score(y_test, y_test_predict)
result_R2.loc[len(result_R2.index)] = ['sklearnLasso', rmse_train, rmse_test]


elastic = MyElasticNet()
elastic.fit(X_train, y_train)
y_train_predict = elastic.predict(X_train)
y_test_predict = elastic.predict(X_test)
rmse_train = myr2(y_train_predict, y_train)
rmse_test = myr2(y_test_predict, y_test)
result_R2.loc[len(result_R2.index)] = ['myElastic', rmse_train, rmse_test]

elastic = ElasticNet()
elastic.fit(X_train, y_train)
y_train_predict = elastic.predict(X_train)
y_test_predict = elastic.predict(X_test)
rmse_train = metrics.r2_score(y_train, y_train_predict)
rmse_test = metrics.r2_score(y_test, y_test_predict)
result_R2.loc[len(result_R2.index)] = ['sklearnElastic', rmse_train, rmse_test]

result_R2

,model_R2,train,test
0,myAnaliticLinearRegression,0.005668,0.337366
1,myGDLinearRegression,0.005668,0.337401
2,mySGDLinearRegression,0.004991,0.382029
3,sklearnLinearRegression,0.005668,0.337366
4,myRidge,0.005668,0.337389
5,sklearnRidge,0.005668,0.337389
6,myLasso,0.005664,0.338571
7,sklearnLasso,0.005667,0.338699
8,myElastic,0.004897,0.346230
9,sklearnElastic,0.004128,0.294906


## 6. Feature normalization

### 6.1. First of all, please write several examples why and where features normalization is mandatory and vice versa.

```Когда нормализация признаков обязательна:```  
1. Когда признаки имеют разные масштабы: Например, если у вас есть признак, измеряемый в миллиметрах, и другой признак, измеряемый в килметрахх, то без нормализации признаков модель может неправильно интерпретировать важность этих признаков из-за их разных масштабов.
2. В алгоритмах, чувствительных к масштабу: Некоторые алгоритмы, такие как k-ближайших соседей (k-NN) и линейная регрессия, могут значительно снизить свою точность, если признаки имеют разные масштабы. Нормализация признаков помогает этим алгоритмам работать более эффективно  
3. При использовании градиентного спуска: В контексте обучения с учителем, таких как логистическая регрессия или нейронные сети, нормализация признаков может ускорить сходимость алгоритма обучения, поскольку градиентный спуск может быстрее сходиться к оптимальному решению.  

```Когда нормализация признаков может быть не нужна:```  
1. Когда признаки уже находятся в одинаковом масштабе: Если все признаки уже находятся в одном и том же масштабе, то дополнительная нормализация может быть не нужна и даже может привести к потере важной инфорции.  
2. В задачах, где масштаб признаков не влияет на результат: В некоторых типах задач, таких как классификация с использованием деревьев решений, масштаб признаков может не иметь значительного влияния на результат. Деревья решений могут автоматически учитывать важность признаков без необходимости их нормалации

### 6.2. Here let's consider the first of classical normalization methods - MinMaxScaler. Please write a mathematical formula for the method.

$$
X_{scaled} = a + \frac{{(X - X_{min})(b-a)}}{X_{max} - X_{min}}
$$
$$ X_{max} - макс.признака $$
$$ X_{min} - мин.признака $$
$$ [a,b] - новый \ диапазон \ значений $$

### 6.3 Implement your own function for MinMaxScaler feature normalization

In [167]:
def MyMinMaxScaler(X, a=0, b=1):
    X_scaled = X.copy()
    for column in X.columns:
        X_scaled[column] = a + (X[column] - X[column].min())*(b - a) / (X[column].max() - X[column].min())
    return X_scaled

### 6.4 Initialize MinMaxScaler() from sklearn.preprocessing.

In [168]:
minmax = MinMaxScaler()

### 6.5 Compare feature normalization with your own method and from sklearn.

In [169]:
MyMinMaxScaler(X_train)

,bathrooms,bedrooms,interest_level,Elevator,CatsAllowed,HardwoodFloors,DogsAllowed,Doorman,Dishwasher,NoFee,...,Pre-War,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
14019,0.25,0.500,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12623,0.10,0.125,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
27405,0.10,0.000,0.5,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7657,0.10,0.125,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7912,0.10,0.125,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16432,0.10,0.125,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8964,0.20,0.375,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5944,0.10,0.250,0.5,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5327,0.10,0.375,0.5,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
minmax.fit_transform(X_train)

array([[0.25 , 0.5  , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.1  , 0.125, 0.   , ..., 0.   , 0.   , 1.   ],
       [0.1  , 0.   , 0.5  , ..., 0.   , 0.   , 1.   ],
       ...,
       [0.1  , 0.25 , 0.5  , ..., 0.   , 0.   , 0.   ],
       [0.1  , 0.375, 0.5  , ..., 0.   , 0.   , 0.   ],
       [0.1  , 0.375, 1.   , ..., 0.   , 0.   , 0.   ]])

In [171]:
metrics.mean_absolute_error(MyMinMaxScaler(X_train), minmax.fit_transform(X_train))

5.691332186896839e-20

### 6.6 Repeat steps from b to e for one more normalization method StandardScaler.

$$
x_{scaled} = \frac{x - \mu}{\sigma}
$$
Где:
$$ \mu - среднее \ арифметическое \ признака $$
$$ \sigma - среднеквадратическое(стандартное) \ отклонение $$

In [172]:
def MyStandartScaler(X):
    X_scaled = X.copy()
    for column in X.columns:
        X_scaled[column] =(X[column] - X[column].mean())/X[column].std()
    return X_scaled

In [173]:
stdscl = StandardScaler()

In [174]:
MyStandartScaler(X_train)

,bathrooms,bedrooms,interest_level,Elevator,CatsAllowed,HardwoodFloors,DogsAllowed,Doorman,Dishwasher,NoFee,...,Pre-War,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
14019,2.564829,2.203708,-0.610460,0.956384,1.046268,1.048874,1.112139,1.167461,1.192523,1.315917,...,-0.477233,-0.390845,-0.402180,2.948007,-0.307907,-0.250373,-0.239332,-0.237255,-0.231482,-0.221029
12623,-0.421987,-0.485465,-0.610460,0.956384,1.046268,-0.953379,1.112139,1.167461,-0.838537,-0.759907,...,-0.477233,-0.390845,2.486388,-0.339204,-0.307907,-0.250373,-0.239332,-0.237255,-0.231482,4.524181
27405,-0.421987,-1.381856,0.987999,0.956384,1.046268,1.048874,1.112139,-0.856538,1.192523,1.315917,...,-0.477233,2.558495,2.486388,-0.339204,3.247655,-0.250373,-0.239332,-0.237255,-0.231482,4.524181
7657,-0.421987,-0.485465,-0.610460,0.956384,1.046268,-0.953379,1.112139,1.167461,-0.838537,-0.759907,...,-0.477233,-0.390845,-0.402180,-0.339204,-0.307907,-0.250373,-0.239332,-0.237255,-0.231482,-0.221029
7912,-0.421987,-0.485465,-0.610460,0.956384,-0.955754,1.048874,-0.899146,-0.856538,-0.838537,-0.759907,...,-0.477233,-0.390845,-0.402180,-0.339204,-0.307907,-0.250373,-0.239332,-0.237255,-0.231482,-0.221029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16432,-0.421987,-0.485465,-0.610460,-1.045578,-0.955754,1.048874,-0.899146,-0.856538,-0.838537,-0.759907,...,-0.477233,-0.390845,-0.402180,-0.339204,-0.307907,-0.250373,-0.239332,-0.237255,-0.231482,-0.221029
8964,1.569224,1.307317,2.586457,0.956384,-0.955754,1.048874,-0.899146,-0.856538,1.192523,1.315917,...,-0.477233,-0.390845,2.486388,-0.339204,-0.307907,3.993934,-0.239332,-0.237255,-0.231482,-0.221029
5944,-0.421987,0.410926,0.987999,0.956384,-0.955754,-0.953379,-0.899146,1.167461,1.192523,1.315917,...,-0.477233,-0.390845,-0.402180,-0.339204,-0.307907,-0.250373,-0.239332,-0.237255,-0.231482,-0.221029
5327,-0.421987,1.307317,0.987999,0.956384,1.046268,1.048874,1.112139,-0.856538,1.192523,-0.759907,...,-0.477233,-0.390845,-0.402180,-0.339204,-0.307907,-0.250373,-0.239332,-0.237255,-0.231482,-0.221029


In [175]:
stdscl.fit_transform(X_train)

array([[ 2.56486173,  2.20373628, -0.6104674 , ..., -0.23725788,
        -0.23148488, -0.22103168],
       [-0.42199212, -0.48547088, -0.6104674 , ..., -0.23725788,
        -0.23148488,  4.52423839],
       [-0.42199212, -1.38187327,  0.9880114 , ..., -0.23725788,
        -0.23148488,  4.52423839],
       ...,
       [-0.42199212,  0.41093151,  0.9880114 , ..., -0.23725788,
        -0.23148488, -0.22103168],
       [-0.42199212,  1.3073339 ,  0.9880114 , ..., -0.23725788,
        -0.23148488, -0.22103168],
       [-0.42199212,  1.3073339 ,  2.58649019, ..., -0.23725788,
        -0.23148488, -0.22103168]])

In [176]:
metrics.mean_absolute_error(stdscl.fit_transform(X_train), MyStandartScaler(X_train))

9.616170117614161e-06

## 7. Fit models with normalization

### 7.1. Fit all models - Linear regression, Ridge, Lasso and ElasticNet with MinMaxScaler.

In [177]:
# Нормализуем данные и разделяем
X_mms = MyMinMaxScaler(X)
# y_mms = MyMinMaxScaler(y)
X_train_mms, X_test_mms, y_train_mms, y_test_mms = train_test_split(X_mms, y, test_size=0.2, random_state=21)

In [178]:
lr = myAnaliticLinearRegression()
lr.fit(X_train_mms, y_train_mms)
y_train_predict_lr_mms = lr.predict(X_train_mms)
y_test_predict_lr_mms = lr.predict(X_test_mms)

l2 = MyRidge()
l2.fit(X_train_mms, y_train_mms)
y_train_predict_l2_mms = l2.predict(X_train_mms)
y_test_predict_l2_mms = l2.predict(X_test_mms)

l1 = MyLasso()
l1.fit(X_train_mms, y_train_mms)
y_train_predict_l1_mms = l1.predict(X_train_mms)
y_test_predict_l1_mms = l1.predict(X_test_mms)

el = MyElasticNet()
el.fit(X_train_mms, y_train_mms)
y_train_predict_el_mms = el.predict(X_train_mms)
y_test_predict_el_mms = el.predict(X_test_mms)

## 7.2 Fit all models - Linear regression, Ridge, Lasso and ElasticNet with StandardScaler.

In [179]:
# Нормализуем данные и разделяем
X_ss = MyStandartScaler(X)
# y_ss = MyStandartScaler(y)
X_train_ss, X_test_ss, y_train_ss, y_test_ss = train_test_split(X_ss, y, test_size=0.2, random_state=21)

In [180]:
lr = myAnaliticLinearRegression()
lr.fit(X_train_ss, y_train_ss)
y_train_predict_lr_ss = lr.predict(X_train_ss)
y_test_predict_lr_ss = lr.predict(X_test_ss)

l2 = MyRidge()
l2.fit(X_train_ss, y_train_ss)
y_train_predict_l2_ss = l2.predict(X_train_ss)
y_test_predict_l2_ss = l2.predict(X_test_ss)

l1 = MyLasso()
l1.fit(X_train_ss, y_train_ss)
y_train_predict_l1_ss = l1.predict(X_train_ss)
y_test_predict_l1_ss = l1.predict(X_test_ss)

el = MyElasticNet()
el.fit(X_train_ss, y_train_ss)
y_train_predict_el_ss = el.predict(X_train_ss)
y_test_predict_el_ss = el.predict(X_test_ss)

## 7.3 Add all results to our dataframe with metrics on samples.

In [181]:
train = myMAE(y_train_predict_lr_mms, y_train_mms)
test = myMAE(y_test_predict_lr_mms, y_test_mms)
result_MAE.loc[len(result_MAE.index)] = ['myAnaliticLinearRegression_MinMaxScaler', train, test]

train = myMAE(y_train_predict_l2_mms, y_train_mms)
test = myMAE(y_test_predict_l2_mms, y_test_mms)
result_MAE.loc[len(result_MAE.index)] = ['myRidge_MinMaxScaler', train, test]

train = myMAE(y_train_predict_l1_mms, y_train_mms)
test = myMAE(y_test_predict_l1_mms, y_test_mms)
result_MAE.loc[len(result_MAE.index)] = ['myLasso_MinMaxScaler', train, test]

train = myMAE(y_train_predict_el_mms, y_train_mms)
test = myMAE(y_test_predict_el_mms, y_test_mms)
result_MAE.loc[len(result_MAE.index)] = ['myElastic_MinMaxScaler', train, test]


train = myMAE(y_train_predict_lr_ss, y_train_ss)
test = myMAE(y_test_predict_lr_ss, y_test_ss)
result_MAE.loc[len(result_MAE.index)] = ['myAnaliticLinearRegression_StandartScaler', train, test]

train = myMAE(y_train_predict_l2_ss, y_train_ss)
test = myMAE(y_test_predict_l2_ss, y_test_ss)
result_MAE.loc[len(result_MAE.index)] = ['myRidge_StandartScaler', train, test]

train = myMAE(y_train_predict_l1_ss, y_train_ss)
test = myMAE(y_test_predict_l1_ss, y_test_ss)
result_MAE.loc[len(result_MAE.index)] = ['myLasso_StandartScaler', train, test]

train = myMAE(y_train_predict_el_ss, y_train_ss)
test = myMAE(y_test_predict_el_ss, y_test_ss)
result_MAE.loc[len(result_MAE.index)] = ['myElastic_StandartScaler', train, test]

result_MAE

,model_MAE,train,test
0,myAnaliticLinearRegression,1229.309643,1028.484374
1,myGDLinearRegression,1229.200764,1028.315910
2,mySGDLinearRegression,1123.739557,917.855819
3,sklearnLinearRegression,1229.309643,1028.484374
4,myRidge,1229.240889,1028.408325
5,sklearnRidge,1229.240889,1028.408325
6,myLasso,1226.315987,1025.342315
7,sklearnLasso,1225.253433,1023.900827
8,myElastic,1108.552334,900.520436
9,sklearnElastic,1150.604032,934.382583


In [182]:
train = myMSE(y_train_predict_lr_mms, y_train_mms)
test = myMSE(y_test_predict_lr_mms, y_test_mms)
result_RMSE.loc[len(result_RMSE.index)] = ['myAnaliticLinearRegression_MinMaxScaler', train, test]

train = myMSE(y_train_predict_l2_mms, y_train_mms)
test = myMSE(y_test_predict_l2_mms, y_test_mms)
result_RMSE.loc[len(result_RMSE.index)] = ['myRidge_MinMaxScaler', train, test]

train = myMSE(y_train_predict_l1_mms, y_train_mms)
test = myMSE(y_test_predict_l1_mms, y_test_mms)
result_RMSE.loc[len(result_RMSE.index)] = ['myLasso_MinMaxScaler', train, test]

train = myMSE(y_train_predict_el_mms, y_train_mms)
test = myMSE(y_test_predict_el_mms, y_test_mms)
result_RMSE.loc[len(result_RMSE.index)] = ['myElastic_MinMaxScaler', train, test]


train = myMSE(y_train_predict_lr_ss, y_train_ss)
test = myMSE(y_test_predict_lr_ss, y_test_ss)
result_RMSE.loc[len(result_RMSE.index)] = ['myAnaliticLinearRegression_StandartScaler', train, test]

train = myMSE(y_train_predict_l2_ss, y_train_ss)
test = myMSE(y_test_predict_l2_ss, y_test_ss)
result_RMSE.loc[len(result_RMSE.index)] = ['myRidge_StandartScaler', train, test]

train = myMSE(y_train_predict_l1_ss, y_train_ss)
test = myMSE(y_test_predict_l1_ss, y_test_ss)
result_RMSE.loc[len(result_RMSE.index)] = ['myLasso_StandartScaler', train, test]

train = myMSE(y_train_predict_el_ss, y_train_ss)
test = myMSE(y_test_predict_el_ss, y_test_ss)
result_RMSE.loc[len(result_RMSE.index)] = ['myElastic_StandartScaler', train, test]

result_RMSE

,model_RMSE,train,test
0,myAnaliticLinearRegression,24565.181235,2175.129388
1,myGDLinearRegression,24565.181401,2175.071869
2,mySGDLinearRegression,24576.992389,2155.258782
3,sklearnLinearRegression,24565.181235,2175.129388
4,myRidge,24565.181236,2175.091064
5,sklearnRidge,24565.181236,2175.091064
6,myLasso,24565.228265,2173.152569
7,sklearnLasso,24565.189431,2172.939434
8,myElastic,24574.694627,2160.532264
9,sklearnElastic,24584.196502,2243.735846


In [183]:
train = myr2(y_train_predict_lr_mms, y_train_mms)
test = myr2(y_test_predict_lr_mms, y_test_mms)
result_R2.loc[len(result_R2.index)] = ['myAnaliticLinearRegression_MinMaxScaler', train, test]

train = myr2(y_train_predict_l2_mms, y_train_mms)
test = myr2(y_test_predict_l2_mms, y_test_mms)
result_R2.loc[len(result_R2.index)] = ['myRidge_MinMaxScaler', train, test]

train = myr2(y_train_predict_l1_mms, y_train_mms)
test = myr2(y_test_predict_l1_mms, y_test_mms)
result_R2.loc[len(result_R2.index)] = ['myLasso_MinMaxScaler', train, test]

train = myr2(y_train_predict_el_mms, y_train_mms)
test = myr2(y_test_predict_el_mms, y_test_mms)
result_R2.loc[len(result_R2.index)] = ['myElastic_MinMaxScaler', train, test]


train = myr2(y_train_predict_lr_ss, y_train_ss)
test = myr2(y_test_predict_lr_ss, y_test_ss)
result_R2.loc[len(result_R2.index)] = ['myAnaliticLinearRegression_StandartScaler', train, test]

train = myr2(y_train_predict_l2_ss, y_train_ss)
test = myr2(y_test_predict_l2_ss, y_test_ss)
result_R2.loc[len(result_R2.index)] = ['myRidge_StandartScaler', train, test]

train = myr2(y_train_predict_l1_ss, y_train_ss)
test = myr2(y_test_predict_l1_ss, y_test_ss)
result_R2.loc[len(result_R2.index)] = ['myLasso_StandartScaler', train, test]

train = myr2(y_train_predict_el_ss, y_train_ss)
test = myr2(y_test_predict_el_ss, y_test_ss)
result_R2.loc[len(result_R2.index)] = ['myElastic_StandartScaler', train, test]

result_R2

,model_R2,train,test
0,myAnaliticLinearRegression,0.005668,0.337366
1,myGDLinearRegression,0.005668,0.337401
2,mySGDLinearRegression,0.004991,0.382029
3,sklearnLinearRegression,0.005668,0.337366
4,myRidge,0.005668,0.337389
5,sklearnRidge,0.005668,0.337389
6,myLasso,0.005664,0.338571
7,sklearnLasso,0.005667,0.338699
8,myElastic,0.004897,0.346230
9,sklearnElastic,0.004128,0.294906


## 10. Compare results

### 10.1 Print your final tables


In [200]:
result_MAE

,model_MAE,train,test
0,myAnaliticLinearRegression,1229.309643,1028.484374
1,myGDLinearRegression,1229.200764,1028.315910
2,mySGDLinearRegression,1123.739557,917.855819
3,sklearnLinearRegression,1229.309643,1028.484374
4,myRidge,1229.240889,1028.408325
5,sklearnRidge,1229.240889,1028.408325
6,myLasso,1226.315987,1025.342315
7,sklearnLasso,1225.253433,1023.900827
8,myElastic,1108.552334,900.520436
9,sklearnElastic,1150.604032,934.382583


In [201]:
result_RMSE

,model_RMSE,train,test
0,myAnaliticLinearRegression,24565.181235,2175.129388
1,myGDLinearRegression,24565.181401,2175.071869
2,mySGDLinearRegression,24576.992389,2155.258782
3,sklearnLinearRegression,24565.181235,2175.129388
4,myRidge,24565.181236,2175.091064
5,sklearnRidge,24565.181236,2175.091064
6,myLasso,24565.228265,2173.152569
7,sklearnLasso,24565.189431,2172.939434
8,myElastic,24574.694627,2160.532264
9,sklearnElastic,24584.196502,2243.735846


In [202]:
result_R2

,model_R2,train,test
0,myAnaliticLinearRegression,0.005668,0.337366
1,myGDLinearRegression,0.005668,0.337401
2,mySGDLinearRegression,0.004991,0.382029
3,sklearnLinearRegression,0.005668,0.337366
4,myRidge,0.005668,0.337389
5,sklearnRidge,0.005668,0.337389
6,myLasso,0.005664,0.338571
7,sklearnLasso,0.005667,0.338699
8,myElastic,0.004897,0.346230
9,sklearnElastic,0.004128,0.294906


### 10.2 What is the best model?

Лучшая можедб на мой взгляд по R^2: myLasso_MinMaxScaler

### 10.3 What is the most stable model?

Наиболее стабильная myLasso